## Generate negative pairs

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
import datasets
from transformers import AutoConfig, AutoModel, AutoTokenizer

MODEL_NAME_OR_PATH="bert-base-uncased"  
MODEL_OUTPUT_PATH = "models/bert-base-uncased"

model = AutoModel.from_pretrained(MODEL_NAME_OR_PATH).cuda()
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME_OR_PATH)

def get_bert_embedding(sentence, model, tokenizer):
    inputs = tokenizer(sentence, return_tensors="pt")
    for key in inputs.keys():
        inputs[key] = inputs[key].cuda()
    outputs = model(**inputs)
    last_hidden_states = outputs.last_hidden_state
    return last_hidden_states[0][0].detach().cpu().numpy()

def get_excluded_effects_index(df_pos, target_index):
    target_wiki_page = df_pos.iloc[target_index]['wiki_page']
    excluded_effects_index = df_pos[df_pos['wiki_page'] == target_wiki_page].index.tolist()
    return excluded_effects_index

df_train_pos = pd.read_csv('../Dataset/EconNLI_train_tmp.csv')

# generate negative examples

causes_embeddings = []
effects_embeddings = []

for row in tqdm(df_train_pos.iterrows(), total=len(df_train_pos)):
    cause_embedding = get_bert_embedding(row[1]['cause'], model, tokenizer)
    effect_embedding = get_bert_embedding(row[1]['effect'], model, tokenizer)
    causes_embeddings.append(cause_embedding)
    effects_embeddings.append(effect_embedding)

causes_embeddings = np.array(causes_embeddings)
effects_embeddings = np.array(effects_embeddings)

cos_mat = cosine_similarity(causes_embeddings, effects_embeddings)

assert cos_mat.shape == (len(df_train_pos), len(df_train_pos))

neg_list = []

for row in tqdm(df_train_pos.iterrows(), total=len(df_train_pos)):
    wiki_page = row[1]['wiki_page']
    sentence = row[1]['sentence']
    cause = row[1]['cause']
    effects_candidates = cos_mat[row[0]].argsort()[-20:]
    excluded_effects_index = get_excluded_effects_index(df_train_pos, row[0])
    effects_candidates = np.setdiff1d(effects_candidates, excluded_effects_index)   # avoid choosing the true effect
    if len(effects_candidates) == 0:
        print('no candidates for row {}'.format(row[0]))
        effects_candidates = cos_mat[row[0]].argsort()[-40:]
        effects_candidates = np.setdiff1d(effects_candidates, excluded_effects_index)
    choice = np.random.choice(effects_candidates)
    neg_effect = df_train_pos.iloc[choice]['effect']
    neg_list.append({'wiki_page': wiki_page, 'sentence': sentence, 'cause': cause, 'effect': neg_effect, 'ChatGPT_label': 0})
    
df_neg = pd.DataFrame(neg_list)
df_train_new = pd.concat([df_train_pos, df_neg], ignore_index=True)

df_train_new.to_csv('../Dataset/EconNLI_train_tmp.csv')

## Assign labels by GPT-4


In [ ]:
import pandas as pd
from tqdm import tqdm
import openai
import backoff
import time

openai.api_key = '' # put your openai api key here

def prompt_gpt(prompt,model_name="gpt-3.5-turbo"): 
    # model_name in ["gpt-3.5-turbo", "gpt-4", "gpt-4o"]
    try:
        completion = openai.ChatCompletion.create(
        model=model_name,
        messages=[
            {"role": "user", "content":prompt}
        ])
    except Exception as e:
        print(str(e))
        time.sleep(6)
        completion = openai.ChatCompletion.create(
        model=model_name,
        messages=[
            {"role": "user", "content":prompt}
        ])
    return completion["choices"][0]["message"]["content"]


@backoff.on_exception(backoff.expo, openai.error.RateLimitError)
def prompt_gpt_with_backoff(prompt,model_name="gpt-3.5-turbo"):
    return prompt_gpt(prompt, model_name)


def relation_classification_via_gpt(cause, effect,model_name, sentence=None):
    if sentence is None:
        prompt = f"Can we infer that {cause} is a cause of {effect} ? Answer Yes or No."
    else:
        prompt = f" Given the sentence {sentence}, can we infer that {cause} is a cause of {effect} ? Answer Yes or No."  
    response = prompt_gpt_with_backoff(prompt, model_name)
    if response.strip("\n").strip(".").strip().lower() == "yes":
        return 1
    elif response.strip("\n").strip(".").strip().lower() == "no":
        return 0
    else:
        return -1

if __name__ == "__main__":

    train_df = pd.read_csv("../Dataset/EconNLI_train_tmp.csv",index_col=0)

    for row in tqdm(train_df.iterrows(),total = len(train_df)):
        results_GPT4 = []
        if row[1]["ChatGPT_label"] == 1:
            result_GPT4 = relation_classification_via_gpt(row[1]["cause"], row[1]["effect"], "gpt-4" ,sentence=row[1]["sentence"])
        else:
            result_GPT4 = relation_classification_via_gpt(row[1]["cause"], row[1]["effect"], "gpt-4")
        results_GPT4.append(result_GPT4)
        train_df.loc[row[0], "GPT4_label"] = results_GPT4
        # time.sleep(2)  # avoid rate limit

    train_df = train_df[~((train_df["ChatGPT_label"] == 0) & (train_df["GPT4_label"] == 1))]
    train_df.to_csv("../Dataset/EconNLI_train_final.csv",index=False)